## Error Correcting Codes

In this lab, we use matrix multiplication to explore the ideas behind hamming codes and their use in error-correction. We use a **generator** matrix in order to represent a vector space. In particular, a generator matrix for a linear code is a matrix whose columns are generators for the set of codewords $\mathcal{C}$.

For some vector $p$, we see by the linear-combination form of matrix multiplication that $G * p$ is a linear combination of the columns of $G$ and is therefore in $\mathcal{C}$ and so it is a codeword.

In [1]:
'''
Initializaiton
'''

from matrix import transpose, mv, mm
from vector import scale, dot, zero_vec, add

def gfadd(u, v):
    '''
    gfadd(u, v) performs vector addition under the GF(2) field

    Parameters:
        u, v (list): n-vectors under GF(2)

    Output:
        An n-vector that is the result of [u] + [v]

    Example:
        u = [1, 0, 1]
        v = [1, 1, 0]

        gfadd(u, v)
        => [0, 1, 1]
    '''
    return [u[k] ^ v[k] for k in range(len(u))]

def gfdot(u, v):
    '''
    gfdot(u, v) computes the dot product of [u] and [v], ensuring
    [u], [v] are treated as vectors in GF(2)

    Parameters:
        u, v (list): n-vectors in GF(2)

    Output:
        A scalar in GF(2)

    Example:
        u = [1, 0, 1, 1]
        v = [1, 1, 1, 0]

        gfdot(u, v)
        => (1 * 1) + (0 * 1) + (1 * 1) + (0 * 1)
        => 1 + 1
        => 0
    '''
    prod = [u[k] * v[k] for k in range(len(u))]

    sum = 0
    for b in prod:
        sum = sum ^ b

    return sum

gfmv = mv(gfadd)

# Generator Matrix
G = [
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [0, 0, 0, 1],
    [1, 1, 1, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
]

# Check Matrix
H = [
    [0, 0, 0, 1, 1, 1, 1],
    [0, 1, 1, 0, 0, 1, 1],
    [1, 0, 1, 0, 1, 0, 1]
]

### Encoding

A 4-bit message $p$ is encoded by $f_G(p) = G * p$, the image of which is a 7-vector. For example we compute the encoding of $p = [1, 0, 0, 1]$:

$$
G =
\begin{bmatrix}
1 & 0 & 1 & 1 \\
1 & 1 & 0 & 1 \\
0 & 0 & 0 & 1 \\
1 & 1 & 1 & 0 \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 \\
1 & 0 & 0 & 0
\end{bmatrix} *
p = [1, 0, 0, 1]
$$

is given by the linear combination

$$
\begin{align}
1 * [1, 1, 0, 1, 0, 0, 1] &+ \\ 
0 * [0, 1, 0, 1, 0, 1, 0] &+ \\ 
0 * [1, 0, 0, 1, 1, 0, 0] &+ \\ 
1 * [1, 1, 1, 0, 0, 0, 0] \\ \\
= [0, 0, 1, 1, 0, 0, 1] 
\end{align}
$$

In [2]:
# Encoding
p = [1, 0, 0, 1]

gfmv(G, p)

[0, 0, 1, 1, 0, 0, 1]

## Manual Decoding

We see that the rows of $G$ correspondng to the standard basis vectors of $GF(2)^4$:

Row 3 = $[0, 0, 0, 1] = e_4$
Row 5 = $[0, 0, 1, 0] = e_3$
Row 6 = $[0, 1, 0, 0] = e_2$
Row 7 = $[1, 0, 0, 0] = e_1$

which suggests: (my guess: That a word $p$ can be written as a linear combination of some of the rows of $G$?)

> Not actually certain what the book means to imply here.

### Example

Decoding $[0, 1, 1, 1, 1, 0, 0]$ by hand:

Using the linear combination defintion of matrix-vector multiplication:

$$
G =
\begin{bmatrix}
1 & 0 & 1 & 1 \\
1 & 1 & 0 & 1 \\
0 & 0 & 0 & 1 \\
1 & 1 & 1 & 0 \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 \\
1 & 0 & 0 & 0
\end{bmatrix} * p
$$

is represented by the linear combination:

$$
\begin{align}
G * p &= p_1[1, 1, 0, 1, 0, 0, 1] \\
&+       p_2[0, 1, 0, 1, 0, 1, 0] \\
&+       p_3[1, 0, 0, 1, 1, 0, 0] \\
&+       p_4[1, 1, 1, 0, 0, 0, 0]
\end{align}
$$

> Not sure if this is the correct method, but it worked for me:

We deduce that $p_1$ must be 0 since the codeword ends in a zero, and no other vector contains a one in the 7th entry. Similarly $p_2$ must be 0 due to the 6th entry of the codeword being 0. Next I guessed that $p_3 = p_4 = 1$ would work out:

$$
[1, 0, 0, 1, 1, 0, 0] + [1, 1, 1, 0, 0, 0, 0] = [0, 1, 1, 1, 1, 0, 0]
$$

So then $p = [0, 0, 1, 1]$

In [3]:
# Verification of decoding algorithm
gfmv(G, [0, 0, 1, 1])

[0, 1, 1, 1, 1, 0, 0]

In [5]:
A = [
    [1, 2],
    [-1, 1]
]

B = [
    [4, 2, 0],
    [3, 1, -1]
]

print(transpose(B))

# mm(dot)(A, B)

[[4, 3], [2, 1], [0, -1]]


## Decoding with the inverse of G

Since $f_G$ is a linear transformation, if it is a bijection, then $G$ is invertible and its inverse would provide a matrix such that $R * c = \text{<original word>}$. We don't yet know how to prove surjectivity of a linear function, but we can try to find an inverse matrix of $G$ regardless.

### Finding the inverse of G

